In [1]:
import pandas as pd
import numpy as np
from gensim.downloader import load as gensim_load
from sklearn.metrics.pairwise import cosine_similarity

# 0. Load pieman_segments
pieman_segments = pd.read_csv("pieman_segments.csv")

# 1. Load pretrained Word2Vec model
model = gensim_load("glove-wiki-gigaword-300")  # 300d

# 2. Define prototype words for each emotion
emotion_words = {
    "happiness": ["happy", "joy", "delighted", "pleased", "content", "excited"],
    "sadness": ["sad", "unhappy", "miserable", "depressed", "gloomy", "tearful"],
    "anger": ["angry", "mad", "furious", "irritated", "annoyed", "enraged"],
    "fear": ["afraid", "scared", "fearful", "terrified", "nervous", "anxious"],
    "surprise": ["surprised", "shocked", "astonished", "amazed", "startled", "stunned"],
    "disgust": ["disgusted", "repulsed", "gross", "nauseated", "revolted", "sickened"]
}

# 3. Compute prototype vectors
emotion_prototypes = {}
for emo, words in emotion_words.items():
    vecs = [model[w] for w in words if w in model]
    if vecs:
        emotion_prototypes[emo] = np.mean(vecs, axis=0)
    else:
        raise ValueError(f"No valid words found in model for emotion: {emo}")

# 4. Process your TR data
tmp = pieman_segments.copy()
tmp["word_raw"] = tmp["word_raw"].astype(str).str.lower()
tmp = tmp[tmp["word_raw"].notna() & tmp["word_raw"].str.len().gt(0)]

words_by_tr = tmp.groupby("TR", as_index=True)["word_raw"].apply(list).sort_index()

# 5. Get average vector per TR
def get_avg_vector(tokens):
    vecs = [model[w] for w in tokens if w in model]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

tr_embeddings = words_by_tr.apply(get_avg_vector)
tr_embed_df = pd.DataFrame(tr_embeddings.tolist(), index=tr_embeddings.index)
tr_embed_df = tr_embed_df.reset_index().rename(columns={i: f"dim_{i}" for i in range(model.vector_size)})

# 6. Compute cosine similarities with each emotion prototype
def cosine_sim(v1, v2):
    return cosine_similarity(v1.reshape(1, -1), v2.reshape(1, -1))[0][0]

for emo, proto_vec in emotion_prototypes.items():
    tr_embed_df[emo] = tr_embed_df[[f"dim_{i}" for i in range(model.vector_size)]].apply(
        lambda row: cosine_sim(row.to_numpy(), proto_vec), axis=1
    )

# 7. Rolling 2-TR trailing mean
all_tr = pd.DataFrame({"TR": np.arange(pieman_segments["TR"].min(), pieman_segments["TR"].max() + 1)})
tr_scores = all_tr.merge(tr_embed_df[["TR"] + list(emotion_prototypes.keys())], on="TR", how="left").fillna(0.0)

In [3]:
# Grab TR by TR for "happiness", "sadness", "anger", "fear", "surprise", "disgust" in TR_scores 
# Also make any rows without a TR (e.g. if we go from 16 to 18 in the TR column) exist and grabbing the previous rows scores for the 6 emo variables
# Train HMM for 2, 3, 4, 5, 6, 7, 8, 9, 10 states (based on the 6 emo variables) and compute log likelihood for each number of states
# Save TR by state csv for best fitting model (BIC)

In [15]:
tr_scores_filled = tr_scores.copy()
tr_scores_filled[list(emotion_prototypes.keys())] = (
    tr_scores_filled[list(emotion_prototypes.keys())]
    .replace(0.0, np.nan)  # Treat 0s as missing
    .ffill()               # Forward-fill missing rows
    .fillna(0.0)           # In case first row is missing
)

In [27]:
from hmmlearn.hmm import GaussianHMM
import time

# 9. Prepare data for HMM
X = tr_scores_filled[emotion_prototypes.keys()].to_numpy()
n_samples, n_features = X.shape

# 10. Fit HMMs for 2 to 10 states, calculate BIC
models = {}
bics = []
log_likelihoods = []

def compute_num_parameters(n_states, n_features):
    p_init  = n_states - 1
    p_trans = n_states * (n_states - 1)
    p_means = n_states * n_features
    p_cov   = n_states * (n_features * (n_features + 1) // 2)
    return p_init + p_trans + p_means + p_cov

for n_states in range(2, 15):
    
    print("Running for State: ", n_states)
    model = GaussianHMM(n_components=n_states, covariance_type="full", random_state=50, n_iter=100, verbose=True)
    model.fit(X)

    logL = model.score(X)
    n_params = compute_num_parameters(n_states, 6)
    bic = -2 * logL + n_params * np.log(n_samples)

    models[n_states] = model
    log_likelihoods.append(logL)
    bics.append(bic)

    time.sleep(2)

Running for State:  2


         1    2479.86646559             +nan
         2    2671.69944234    +191.83297675
         3    2695.00351026     +23.30406792
         4    2707.10748162     +12.10397136
         5    2719.58905676     +12.48157514
         6    2732.68489349     +13.09583673
         7    2738.68525849      +6.00036501
         8    2741.89327424      +3.20801575
         9    2744.09347664      +2.20020240
        10    2745.82565118      +1.73217454
        11    2747.28778253      +1.46213135
        12    2748.47741875      +1.18963622
        13    2749.32464140      +0.84722265
        14    2749.84522615      +0.52058475
        15    2750.13103272      +0.28580657
        16    2750.27940675      +0.14837403
        17    2750.35732055      +0.07791381
        18    2750.40132189      +0.04400134
        19    2750.42898586      +0.02766397
        20    2750.44831426      +0.01932840
        21    2750.46293302      +0.01461875
        22    2750.47454184      +0.01160882
        23

Running for State:  3


         1    2488.51048141             +nan
         2    2705.28853098    +216.77804957
         3    2740.32928769     +35.04075671
         4    2758.27759086     +17.94830317
         5    2766.99693483      +8.71934397
         6    2770.58326167      +3.58632684
         7    2772.59111860      +2.00785693
         8    2774.58569984      +1.99458123
         9    2777.00045163      +2.41475180
        10    2778.52263297      +1.52218134
        11    2779.34155094      +0.81891797
        12    2779.92942426      +0.58787332
        13    2780.43015105      +0.50072678
        14    2780.88128332      +0.45113227
        15    2781.28149495      +0.40021163
        16    2781.62160500      +0.34011005
        17    2781.89917577      +0.27757076
        18    2782.12027867      +0.22110291
        19    2782.29478002      +0.17450134
        20    2782.43244722      +0.13766721
        21    2782.54154126      +0.10909403
        22    2782.62863549      +0.08709424
        23

Running for State:  4


...

         1    2459.67180885             +nan
         2    2782.16317167    +322.49136282
         3    2871.70539386     +89.54222219
         4    2946.64227927     +74.93688541
         5    3013.58841425     +66.94613498
         6    3032.72857931     +19.14016506
         7    3046.72772487     +13.99914556
         8    3147.35200649    +100.62428162
         9    3061.03932082     -86.31268567
Model is not converging.  Current: 3061.03932081978 is not greater than 3147.352006491369. Delta is -86.31268567158895


Running for State:  13


         1    2386.81460437             +nan
         2    2777.51400166    +390.69939729
         3    2865.40295206     +87.88895040
         4    2938.44758197     +73.04462991
         5    3001.87228729     +63.42470531
         6    3071.23619697     +69.36390968
         7    3100.94256764     +29.70637067
         8    3118.48466110     +17.54209347
         9    3152.72622193     +34.24156082
        10    3075.01840458     -77.70781734
Model is not converging.  Current: 3075.0184045836572 is not greater than 3152.7262219262975. Delta is -77.70781734264028


Running for State:  14


         1    2383.35136641             +nan
         2    2762.51874464    +379.16737823
         3    2865.36712182    +102.84837718
         4    2950.92500046     +85.55787864
         5    3105.14590821    +154.22090775
         6    3159.12153559     +53.97562738
         7    3144.75739034     -14.36414524
Model is not converging.  Current: 3144.7573903441285 is not greater than 3159.121535588496. Delta is -14.364145244367592


In [28]:
import matplotlib.pyplot as plt

states_tested = list(range(2, len(bics) + 2))

fig, ax1 = plt.subplots(figsize=(8, 5))

# Plot BIC on left y-axis
color_bic = 'tab:blue'
ax1.set_xlabel("Number of States")
ax1.set_ylabel("BIC", color=color_bic)
ax1.plot(states_tested, bics, marker='o', color=color_bic, label="BIC")
ax1.tick_params(axis='y', labelcolor=color_bic)

# Create right y-axis for log-likelihood
ax2 = ax1.twinx()
color_ll = 'tab:orange'
ax2.set_ylabel("Log-Likelihood", color=color_ll)
ax2.plot(states_tested, log_likelihoods, marker='s', color=color_ll, label="Log-Likelihood")
ax2.tick_params(axis='y', labelcolor=color_ll)

# Title and grid
plt.title("BIC and Log-Likelihood by Number of HMM States")
fig.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
# 11. Select best model by BIC
best_n = range(2, 11)[np.argmin(bics)]
best_model = models[best_n]
print(f"Best model: {best_n} states (BIC = {min(bics):.2f})")

# 12. Predict state sequence
state_sequence = best_model.predict(X)

# 13. Save TR-by-state DataFrame
tr_state_df = tr_scores_filled[["TR"]].copy()
tr_state_df["State"] = state_sequence
tr_state_df.to_csv("pieman_tr_by_state.csv", index=False)